In [1]:
import sys

sys.path.insert(0, '..')

import shutil
import uuid
from verification_service.worker.jobs import Supervisor, Worker
from verification_service.storage.database import MongoDbConnector
from pymongo.mongo_client import MongoClient
from dotenv import load_dotenv
import os 
from functools import partial
from verification_service import unique_id


_outs = './test_outputs'
if os.path.exists(_outs):
    shutil.rmtree(_outs)


def jobid(): return str(uuid.uuid4())


load_dotenv("../verification_service/.env")
uri = os.getenv("MONGO_DB_URI")
omex_source_dir = './examples/sbml-core'
omex_name = 'Elowitz-Nature-2000-Repressilator.omex'
omex_fp = os.path.join(omex_source_dir, omex_name)
out_dir = './test_outputs'
simulators = ['amici', 'copasi', 'tellurium']
spec_name = 'cI mRNA'
job_id = jobid()

db_connector = MongoDbConnector(connection_uri=uri, database_id="service_requests")

Smoldyn is not properly installed in this environment and thus its process implementation cannot be registered. Please consult smoldyn documentation.
Cannot register SimpleNeuron. Error:
**
No module named 'pyneuroml'
**


ImportError: cannot import name 'cascading_load_arrows' from 'verification_service.data_model.worker' (/Users/alexanderpatrie/Desktop/repos/verification-service/verification_service/data_model/worker.py)

In [ ]:
supervisor = Supervisor(db_connector=db_connector)

In [ ]:
supervisor.jobs

In [ ]:
# re-create loop here

import asyncio
from verification_service import load_arrows


db_connector = MongoDbConnector(connection_uri=uri, database_id="service_requests")
supervisor = Supervisor(db_connector=db_connector)
jobs_to_complete = []
        

async def fetch_jobs(supervisor: Supervisor, max_retries=5, delay=5):
    pending_jobs = [job for job in supervisor.db_connector.db['pending_jobs'].find()]
    async def _run_check():
        if len(pending_jobs):
            print('There are pending jobs')
            result = await check_jobs(supervisor)
            print('There are no pending jobs')
            return result
        else:
            return None 
            
    n_retries = 0
    run = True
    while run:
        check = await _run_check()
        if check is None:
            n_retries += 1
            await asyncio.sleep(delay)
        if n_retries == max_retries:
            run = False
        else:
            continue 
    return 0 
        
        
async def check_jobs(supervisor, max_retries=5, delay=5) -> int:
    job_queue = supervisor.pending_jobs
    n_tries = 0
    while True:
        # count tries
        n_tries += 1
        if n_tries > 1:
            await asyncio.sleep(delay)
        elif n_tries == max_retries + 1:
            print(f'Max retries {max_retries} reached!')
            break
        else:
            if len(job_queue):
                print('There are pending jobs.')
                for i, job in enumerate(job_queue):
                    # get the next job in the queue based on the preferred_queue_index
                    job_doc = supervisor.pending_jobs.pop(supervisor.preferred_queue_index)
                    job_comparison_id = job_doc['comparison_id']
                    unique_id_query = {'comparison_id': job_comparison_id}
                    in_progress_job = supervisor.db_connector.db.in_progress_jobs.find_one(unique_id_query) or None
                    
                    _job_exists = partial(supervisor._job_exists, comparison_id=job_comparison_id)
                    if not _job_exists(collection_name='in_progress_jobs'):
                        print(f"In progress job does not yet exist for {job_comparison_id}")
                        in_progress_job_id = unique_id()
                        worker_id = unique_id()
                        id_kwargs = ['job_id', 'worker_id']
                        in_prog_kwargs = dict(zip(
                            id_kwargs,
                            list(map(lambda k: unique_id(), id_kwargs))
                        ))
                        in_prog_kwargs['comparison_id'] = job_comparison_id
                        
                        supervisor.db_connector.insert_in_progress_job(**in_prog_kwargs)
                        print(f"Successfully created new progress job for {job_comparison_id}")
                        # await supervisor.async_refresh_jobs()
                        
                    if not _job_exists(collection_name='completed_jobs'):
                        print(f"Completed job does not yet exist for {job_comparison_id}")
                        # pop in-progress job from internal queue and use it parameterize the worker
                        in_prog_id = [job for job in db_connector.in_progress_jobs.find()].pop(supervisor.preferred_queue_index)['job_id']
                        
                        # double-check and verify doc
                        in_progress_doc = supervisor.db_connector.db.in_progress_jobs.find_one({'job_id': in_prog_id})
                        
                        # generate new worker
                        workers_id = in_progress_doc['worker_id']
                        worker = supervisor.call_worker(job_params=job_doc, worker_id=workers_id)
                        
                        # add the worker to the list of workers (for threadsafety)
                        supervisor.workers.insert(supervisor.preferred_queue_index, worker.worker_id)
                        
                        # the worker returns the job result to the supervisor who saves it as part of a new completed job in the database
                        completed_doc = supervisor.db_connector.insert_completed_job(job_id=unique_id(), comparison_id=job_comparison_id, results=worker.job_result)
                        
                        # release the worker from being busy and refresh jobs
                        supervisor.workers.pop(supervisor.preferred_queue_index)
                        print(f"Successfully created new completed job for {job_comparison_id}")
                        # await supervisor.async_refresh_jobs()
                
                    # remove the job from queue
                    supervisor.pending_jobs.pop(i)
        # sleep
        print(f'Sleeping for {delay} seconds...')
        await load_arrows(delay)
                 
    return 0

In [ ]:
result = await check_jobs(supervisor, max_retries=5, delay=3)

In [ ]:
supervisor.pending_jobs

In [ ]:
supervisor.check_jobs()

In [ ]:
# 1. get an unassigned pending job by id
job_id = supervisor.jobs['pending_jobs'].pop(0)

In [ ]:
# 2. fetch the respective document/job
job_doc = supervisor.db_connector.db.pending_jobs.find_one({'job_id': job_id})

In [ ]:
job_doc

In [ ]:
job_params = job_doc.copy()

In [ ]:
import os 
from verification_service import unique_id

os.path.exists(job_params['omex_path'])

In [ ]:
# 3. Create a new in process job for the pending job we just picked up
worker_id = unique_id()
in_progress_job_id = unique_id()
in_progress_doc = supervisor.db_connector.insert_in_progress_job(
    job_id=in_progress_job_id,
    worker_id=worker_id,
    comparison_id=job_doc['comparison_id'],
)

in_progress_doc

In [ ]:
# 4. Call the worker who will automatically process the job
worker = Worker(job_params=job_params)

In [ ]:
# 4. Get the result from the worker and insert the new completed job for that comparison_id
from verification_service import unique_id

comparison_id = job_doc['comparison_id']

completed_doc = supervisor.db_connector.insert_completed_job(
    job_id=unique_id(),
    comparison_id=comparison_id,
    results=worker.job_result
)

In [ ]:
completed_doc

### Test `Supervisor.initialize()`

In [ ]:
from functools import partial

# case: uncompleted/pending jobs exist
jobs_to_complete = pending
if len(pending):
    in_progress_jobs = supervisor.jobs['in_progress_jobs']
    preferred_queue_index = supervisor.preferred_queue_index  # TODO: How can we make this more robust/dyn
    

In [ ]:
supervisor.get_jobs()